In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

In [ ]:
filename = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Current_G3x_con.csv'
df = pd.read_csv(filename, sep = ',')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('#', 'num')

# df['Net_Shipments'] = df['Net_Shipments'].str.replace(',','')
# df['Tier_1_Objective'] = df['Tier_1_Objective'].str.replace(',','')
# df['Tier_2_Objective'] = df['Tier_2_Objective'].str.replace(',','')
# df['Tier_3_Objective'] = df['Tier_3_Objective'].str.replace(',','')

filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/GDY_AVI_TSN_Incentive.txt'
df_tierMap_TSN = pd.read_csv(filename2, sep = '\t')
df_tierMap_TSN.columns = df_tierMap_TSN.columns.str.replace(' ', '_')

filename3 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/GDY_AVI_NonTSN_Incentive.txt'
df_tierMap_nonTSN = pd.read_csv(filename3, sep = '\t')
df_tierMap_nonTSN.columns = df_tierMap_nonTSN.columns.str.replace(' ', '_')

df.shape

In [ ]:
# df.columns.to_series(index=range(df.shape[1]))

In [ ]:
currentYear = datetime.now().year

In [ ]:
tup_now = datetime.now().timetuple()

In [ ]:
ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData)
# print(dt_currentData)
ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_incentive)

In [ ]:
calculatedData = pd.DataFrame([])
for ii in range(df.shape[0]):

    singleCusty = df.iloc[[ii]]
    singleCusty.reset_index(inplace=True)

    singleCusty_select = singleCusty[
        ['G3X_CO_Name','TSN','Net_Shipments_YTD','num_Locations']]

#     tt = pd.to_datetime(singleCusty['Last_Transaction'])[0].timetuple()
    excel_date = singleCusty.Last_Transaction
    
    date_lt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + excel_date - 2)
    tt = date_lt.timetuple()
        
    if  tt.tm_year == currentYear:

        if (tt.tm_mon == 1) | (tt.tm_mon == 2) | (tt.tm_mon == 3):
            if ((tt.tm_year % 400 == 0) | (tt.tm_year % 4 == 0)) & (tt.tm_year % 100 != 0):
                daysINyear = 366
                print('Leap year')
            else:
                daysINyear = 365           
        
        numDays = tt.tm_yday # get number of days

        estimatedTires = np.floor(singleCusty_select.Net_Shipments_YTD.astype(int)/(numDays/daysINyear)).values
        numLocations = singleCusty.num_Locations.values
#         netShip = singleCusty_select.Net_Shipments.values.astype(int)
#         netGYc = singleCusty_select.avi_Core_Net_Shipments.values.astype(int)
#         netGYm = singleCusty_select.avi_Mid_Net_Shipments.values.astype(int)
#         netKelly = singleCusty_select.Kelly_Net_Shipments.values.astype(int)

        # Check if customer is aligned
        if singleCusty_select.TSN.values == 'N':
            df_tierMap = df_tierMap_nonTSN
            if   (estimatedTires >= (df_tierMap.iloc[0,2] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[0,3] * numLocations)):
                estTier = 0
            else:
                estTier = 1
            if estTier < 1:
                tierUP = estTier + 1
            else:
                tierUP = 1
        else:
            df_tierMap = df_tierMap_TSN
            if   (estimatedTires >= (df_tierMap.iloc[0,2] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[0,3] * numLocations)):
                estTier = 0
            elif (estimatedTires >= (df_tierMap.iloc[1,2] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[1,3] * numLocations)):
                estTier = 1
            else:
                estTier = 2
            if estTier < 2:
                tierUP = estTier + 1
            else:
                tierUP = 2
                
        # floor to underestimate estimated tires
        avi_est_tires_rebate = estimatedTires * df_tierMap.iloc[estTier,1]
        avi_1up_tires = df_tierMap.iloc[tierUP,2]
        avi_1up_tires_rebate = avi_1up_tires * df_tierMap.iloc[tierUP,1]

        if avi_est_tires_rebate >= avi_1up_tires_rebate*numLocations:
            avi_1up_tires = estimatedTires
            avi_1up_tires_rebate = avi_est_tires_rebate

        range_dollars_ll = ((avi_1up_tires*df_tierMap.iloc[0,1]))#.values
        range_dollars_ul = ((avi_1up_tires*df_tierMap.iloc[tierUP,1]))#.values
        
###########################################################################################################################        
#         print(singleCusty_select)
# ###########################################################################################################################     
#         print('\nEstimated Tier:', [estTier])        
#         print('\nEstimated Tires ~ avi:', estimatedTires)        
#         print('\nEstimated Rebate ~ avi:',avi_est_tires_rebate)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################
#         print('Estimated 1up Tier:', [tierUP])        
#         print('\n1up Tires ~ avi:', avi_1up_tires*numLocations)        
#         print('\n1up Dollars ~ avi:',avi_1up_tires_rebate*numLocations)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################        
#         print('Range_ll:',range_dollars_ll)
#         print('Range_ul:',range_dollars_ul*numLocations, '\n')
# ###########################################################################################################################
#         print(tt, '\n' 'indexNumber:', ii, '\n')
###########################################################################################################################

        tempData = pd.DataFrame(
            {'estimated_tier':estTier,
             'current_est_tires':estimatedTires, 'total_est_rebate':avi_est_tires_rebate,
             'Up_tier':tierUP,
             'avi_up_tires':avi_1up_tires*numLocations, 'avi_up_rebate':avi_1up_tires_rebate*numLocations,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul*numLocations,'grain': 'Annual'})
        calculatedData = calculatedData.append(tempData)
    else:
#         print('\nJUNK:', tt,'\n', 'indexNumber:', ii)
        tempData = pd.DataFrame(
            {'estimated_tier':0,
             'current_est_tires':0, 'total_est_rebate':0,
             'Up_tier':1,
             'avi_up_tires':0*numLocations, 'avi_up_rebate':0*numLocations,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul,'grain': 'Annual'})
        calculatedData = calculatedData.append(tempData)
#         print(ii)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df], axis=1)

In [ ]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)
# finalData.head()

In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_G3X_AVI.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_G3X_AVI_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

In [ ]:
# path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
# writeFilename = path + 'cmp_G3X.csv'
# finalData.to_csv(writeFilename, index=False)
# writeFilename_archive = path + 'csv_archive/' + str(dt_currentData.year) + '_' + str(dt_currentData.month) + '_' + str(dt_currentData.day) + '_' + 'CMP_G3X.csv'
# finalData.to_csv(writeFilename_archive, index=False)